<a href="https://colab.research.google.com/github/vitorhugo653/Projetoalura/blob/main/BPL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Suprimir saidas
%%capture
!pip install PyPDF2
!pip install sentence_transformers
!pip install -U -q google.generativeai

from sentence_transformers import SentenceTransformer, util
import PyPDF2
import requests
import google.generativeai as genai

In [ ]:
# Configurar API do Google Generative AI (substitua pela sua chave real)
import os
from google.colab import userdata
api_key = userdata.get('secret_key')
os.environ["GOOGLE_API_KEY"] = api_key
genai.configure()

In [ ]:
# Carregue um modelo de embeddings pré-treinado
model = SentenceTransformer('all-mpnet-base-v2')

In [ ]:
# Função para extrair texto de um PDF a partir de uma URL
def extract_text_from_pdf_url(pdf_url):
  response = requests.get(pdf_url)
  with open('temp.pdf', 'wb') as f:
    f.write(response.content)
  with open('temp.pdf', 'rb') as f:
    pdf_reader = PyPDF2.PdfReader(f)
    text = ""
    for page_num in range(len(pdf_reader.pages)):
      page = pdf_reader.pages[page_num]
      text += page.extract_text()
  return text

In [ ]:
# URLs dos seus PDFs
pdf_urls = [
    'https://antigo.anvisa.gov.br/documents/10181/6278771/RDC_512_2021_.pdf/5650229b-218e-467a-83dd-e292581c20fe',
    'https://www.exactusmetrologia.com.br/sites/default/files/3-nbr_iso_iec_17025-2017_versao_exclusiva_treinamento.pdf'
]

In [ ]:
# Extraia o texto dos PDFs
documents = []
for url in pdf_urls:
    documents.append({
        'url': url,
        'content': extract_text_from_pdf_url(url)
    })

In [ ]:
# Gere embeddings para cada documento
for document in documents:
    document['embedding'] = model.encode(document['content'])

# Função para buscar documentos
def search_documents(query):
    query_embedding = model.encode(query)
    results = []
    for document in documents:
        similarity = util.cos_sim(query_embedding, document['embedding'])
        results.append({
            'url': document['url'],
            'content': document['content'],
            'similarity': similarity.item()
        })
    return sorted(results, key=lambda x: x['similarity'], reverse=True)

In [ ]:
# Função para formatar em Markdown
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
# Função do chatbot
def chatbot():
    chat_history = []  # Armazena o histórico da conversa

    while True:
        query = input("\nFaça sua pergunta:\n")
        if query.lower() == 'sair':
            break

        results = search_documents(query)
        if results:
            best_match = results[0]

            # Reformule a resposta
            prompt = f"Rescreva esse texto de uma maneira descontraída: {best_match['content']}"
            model_2 = genai.GenerativeModel("gemini-pro")
            response = model_2.generate_content(prompt)

            # Adiciona a pergunta e a resposta ao histórico
            chat_history.append({'role': 'Usuário', 'content': query})
            chat_history.append({'role': 'Chatbot', 'content': response.text})

            # Imprime o histórico formatado
            for message in chat_history:
                display(to_markdown(f'**{message["role"]}**: {message["content"]}'))
                print('-'*100)

        else:
            print("Nenhum resultado encontrado.")

# Inicie o chatbot
chatbot()